<a href="https://colab.research.google.com/github/deeplearning-itba/cnn/blob/master/Trabajo_Final_CNN_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [ ]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
#!mkdir /content/output

In [1]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [2]:
Antes que nada importar el siguiente link en tu GDri

SyntaxError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DATASET_PATH = '/content/drive/My Drive/Data_Colab/ArtisticStyle/'
IMAGES_PATH = DATASET_PATH + 'Input/Content/'
STYLE_PATH = DATASET_PATH + 'Input/Style/'
OUTPUT_PATH = DATASET_PATH + 'Output/'

In [5]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida
#775px-Neckarfront_Tübingen_Mai_2017
base_image_path = Path(IMAGES_PATH + "barco2.JPG")
#base_image_path = Path(IMAGES_PATH + "DSC06703.JPG")
#base_image_path = Path(IMAGES_PATH + "DSC06296.JPG")
#base_image_path = Path(IMAGES_PATH + "775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path(STYLE_PATH+"Quinquela-Actividad-en-La-Boca1.jpg")
#style_reference_image_path = Path(STYLE_PATH+"Riachuelo_Quinquela.jpg")
#style_reference_image_path = Path(STYLE_PATH+"La_noche_estrellada1.jpg")
#style_reference_image_path = Path(STYLE_PATH+"16198florencio.jpg")
result_prefix = Path(DATASET_PATH)
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:
https://wandb.ai/authors/nerual_style_transfer/reports/Part-1-Deep-Representations-a-way-towards-neural-style-transfer--VmlldzoyMjQzNDY

Respuesta: 


*  style_weight= beta y es el weight del estilo del arte 
*  content_weight=alpha y es el weigh de la representacion
*  El total_variation_weight es la relación alpha/beta.

Para la prueba de imagen elegida por mi se usó una relación 10-2




In [6]:
style_weight = 1
content_weight = 0.01
total_variation_weight = content_weight/style_weight  #alpha/beta
print(format(total_variation_weight, "10.3E"))

 1.000E-02


In [7]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [8]:
width, height,img_ncols

(852, 639, 533)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:

- 1era linea Carga la imagen con las dimensiones establecidas

- 2nda linea Se pasa a formato array la foto

- 3era linea Se agrega una nueva dimensión 0. La razón es que Keras trabaja con batches de imagenes. Así que la primera dimensión se usa para el number de muestras(o imagenes que se tenga).
https://stackoverflow.com/questions/47555829/preprocess-input-method-in-keras

- 4ta linea Se carga la imagen a la VGG19 al formato.



In [9]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:
- Reconvierte el array insertado en la VGG19 a imagen. (saca la dimensión extra, le carga los promedios por canal e invierte el array para hacerlo RGB)

- Además al subir el modelo VGG19 se cambio pooling por "avg" ya que en el paper indica que mejora la convergencia del gradiente y un resultado un poco mas real.

In [10]:
def deprocess_image (x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [11]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 

In [12]:
base_image = K.variable(preprocess_image(base_image_path))

In [14]:
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [15]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [16]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [17]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False,pooling='avg')
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

80142336/80134624 [==============================] - 4s 0us/step
Model loaded.


# 4) En la siguientes celdas:

1 ¿Qué es la matriz de Gram?¿Para qué se usa?
- Se usa la Gram Matrix para medir el grado de correlacion entre cada uno de los canales que luego se usara como una medicion del tipo de Style en si mismo. Se calcula en base a la dimension de la capa de activación por cada uno de los canales y se usa la transpuesta G= M x M tranpuesta

2 ¿Por qué se permutan las dimensiones de x?
- Sirve para modificar las dimensiones del array imagen y llevarlo para el formato que se ingrea batch_flatten, donde se generar 3 vectores flatten 1D.

https://towardsdatascience.com/neural-style-transfer-tutorial-part-1-f5cd3315fa7f

https://towardsdatascience.com/neural-networks-intuitions-2-dot-product-gram-matrix-and-neural-style-transfer-5d39653e7916




In [18]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

- 1ero Calcula loss etre la Gram Matix menos la Gram Matrix de la imagen generada (MSE)

- 2da Mide la diferencia cuadratica entre los feature de representación en un layer determinado de la imagen original y la imagen generada

- 3ero total_variation_loss es para regularizar los componentes de alta frecuencia de la imagen
https://www.tensorflow.org/tutorials/generative/style_transfer

In [19]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [20]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [21]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [22]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [23]:
grads = K.gradients(loss, combination_image)
#grads = tf.GradientTape(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

- fmin_l_bfgs_b se basa en el algoritmo de memoria limtada BFGS  y se usa para reducir en este caso el minimo de una funcion determinada. 
Se busca la función a minimizar, loss, luego se agrega el vector X incial de la imagen y la función gradiente calculada con Keras.
f_outputs define la loss y el gradiente de la imagen combinada en la primera corrida.
Luego con eval_loss_and_grads y Evaluator class generan la función de loss y gradiente para la fmin_l_bfgs_b.

https://en.wikipedia.org/wiki/Limited-memory_BFGS

- La diferencia que se ve es que no se usó la matriz gram, la loss y el gradiente mostrados en el paper. Se observa que en la impletación se usaron las funciones keras.gradient con la loss de inicio calculada en base al paper y se pasó las dimensiones del layer de combinacion.Y luego, como se hace mension arriba, se usa keras.function() junto con la class Evaluator para calcular al mismo momento la loss y el gradiente.

- Leyenedo el tutorial de Tensorflow se podría usar con GradientTape()

In [24]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [25]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [26]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 3550283500.0
Image saved as /content/drive/My Drive/Data_Colab/ArtisticStyle/output_at_iteration_0.png
Iteration 0 completed in 24s
Start of iteration 1
Current loss value: 1450553600.0
Image saved as /content/drive/My Drive/Data_Colab/ArtisticStyle/output_at_iteration_1.png
Iteration 1 completed in 14s
Start of iteration 2
Current loss value: 965058200.0
Image saved as /content/drive/My Drive/Data_Colab/ArtisticStyle/output_at_iteration_2.png
Iteration 2 completed in 14s
Start of iteration 3
Current loss value: 775842700.0
Image saved as /content/drive/My Drive/Data_Colab/ArtisticStyle/output_at_iteration_3.png
Iteration 3 completed in 14s
Start of iteration 4
Current loss value: 658797250.0
Image saved as /content/drive/My Drive/Data_Colab/ArtisticStyle/output_at_iteration_4.png
Iteration 4 completed in 14s
Start of iteration 5
Current loss value: 593817500.0
Image saved as /content/drive/My Drive/Data_Colab/ArtisticStyle/output_at_iteration_5

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta: